In [1]:
#installing the dependencies required
!pip install accelerate peft bitsandbytes git+https://github.com/huggingface/transformers trl pyzr auto-gptq optimum

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-d70gou1h
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-d70gou1h
  Resolved https://github.com/huggingface/transformers to commit b72752f06830cb6cf8d21c284f68e15faa100c4d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 66.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.4/418.4 kB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/54

In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:

import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM, prepare_model_for_kbit_training, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig, TrainingArguments
from trl import SFTTrainer
import os

data = load_dataset("tatsu-lab/alpaca", split="train")
data_df = data.to_pandas()
data_df = data_df[:5000]
data_df["text"] = data_df[["input", "instruction", "output"]].apply(lambda x: "###Human: " + x["instruction"] + " " + x["input"] + " ###Assistant: "+ x["output"], axis=1)
data = Dataset.from_pandas(data_df)


tokenizer = AutoTokenizer.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.1-GPTQ")
tokenizer.pad_token = tokenizer.eos_token


quantization_config_loading = GPTQConfig(bits=4, disable_exllama=True, tokenizer=tokenizer)
model = AutoModelForCausalLM.from_pretrained(
                            "TheBloke/Mistral-7B-Instruct-v0.1-GPTQ",
                            quantization_config=quantization_config_loading,
                            device_map="auto"
                        )


model.config.use_cache=False
model.config.pretraining_tp=1
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)


peft_config = LoraConfig(
    r=16, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM", target_modules=["q_proj", "v_proj"]
)
model = get_peft_model(model, peft_config)


training_arguments = TrainingArguments(
        output_dir="mistral-finetuned-alpaca",
        per_device_train_batch_size=8,
        gradient_accumulation_steps=1,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=100,
        num_train_epochs=1,
        max_steps=250,
        fp16=True,
        push_to_hub=True
)


trainer = SFTTrainer(
        model=model,
        train_dataset=data,
        peft_config=peft_config,
        dataset_text_field="text",
        args=training_arguments,
        tokenizer=tokenizer,
        packing=False,
        max_seq_length=512
)


trainer.train()


Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.
/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:167: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.However, loading attributes (e.g. ['use_cuda_fp16', 'use_exllama', 'max_input_length', 'exllama_config', 'disable_exllama']) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.
  warnings.warn(warning_msg)
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4492: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use 

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
100,1.423600


Step,Training Loss
100,1.423600
200,1.255300


TrainOutput(global_step=250, training_loss=1.3158250579833983, metrics={'train_runtime': 1386.5674, 'train_samples_per_second': 1.442, 'train_steps_per_second': 0.18, 'total_flos': 311794752946176.0, 'train_loss': 1.3158250579833983, 'epoch': 0.4})

##Mistral Chatbot

Above the k is a arbitrary value above its a 4 bit training model

trained on alpaca data set-which is a famous conversation dataset

LORA -with small decomposition of model training can train the whole model

LORA rank=16 (lower the rank lesser the performance)(scaleable)

Causal LLM other than seq to seq

Target model: v_proj and qproj

getpft model is our base model

paged_adamw_32bit is the optimizer used

learning_rate=2e-4

lr_scheduler_type="cosine",

logging step evaluate at a given step 100 layers=100 steps

max steps:250 * effective batch size 250 =2000 (executed for 2000 steps)

fp16=True which will train the model in mixed depression

In [6]:

! cp -r /content/mistral-finetuned-alpaca /content/drive/MyDrive/

cp: cannot create directory '/content/drive/MyDrive/': No such file or directory


##Inference

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
from peft import AutoPeftModelForCausalLM
from transformers import GenerationConfig
from transformers import AutoTokenizer
import torch
tokenizer = AutoTokenizer.from_pretrained("/content/mistral-finetuned-alpaca")

inputs = tokenizer("""###Human: Why mobile is bad for human?? ###Assistant: """, return_tensors="pt").to("cuda")

In [14]:
model = AutoPeftModelForCausalLM.from_pretrained(
    "/content/mistral-finetuned-alpaca",
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="cuda")

In [10]:

generation_config = GenerationConfig(
    do_sample=True,
    top_k=1,
    temperature=0.1,
    max_new_tokens=100,
    pad_token_id=tokenizer.eos_token_id
)


In [11]:

import time
st_time = time.time()
outputs = model.generate(**inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(time.time()-st_time)

###Human: Why mobile is bad for human? ###Assistant: 1. Mobile devices can cause eye strain and headaches.
2. Mobile devices can cause poor posture and lead to back pain.
3. Mobile devices can cause poor sleep quality.
4. Mobile devices can cause poor memory retention.
5. Mobile devices can cause poor attention span.
6. Mobile devices can cause poor social skills.
7. Mobile devices can cause poor communication skills.
8. Mobile devices can cause poor time management skills.
9. Mobile
106.5844395160675


i/p token-Why mobile is bad for human?


o/p- Mobile devices can cause poor posture and lead to back pain.
3. Mobile devices can cause poor sleep quality.
4. Mobile devices can cause poor memory retention.
5. Mobile devices can cause poor attention span.
6. Mobile devices can cause poor social skills.
7. Mobile devices can cause poor communication skills.
8. Mobile devices can cause poor time management skills.
9. Mobile
106.5844395160675

Used Autotokenizer

Step	Training Loss
100	1.423600
200	1.255300

